In [36]:
!pip install numpy==1.24.4 pandas==1.5.3 openpyxl==3.0.10

  Using cached numpy-1.24.4.tar.gz (10.9 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [38]:
# 必要なら最初にインストール
# %pip install pandas openpyxl

import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Google Colabで使う場合のみ有効化
from google.colab import files

# ① ファイル読み込み＆連結
file_2022 = "2022_年間売上表.xlsx"
file_2023 = "2023_年間売上表.xlsx"

df22 = pd.read_excel(file_2022, sheet_name="Sheet1")
df23 = pd.read_excel(file_2023, sheet_name="Sheet1")
df_all = pd.concat([df22, df23], ignore_index=True)

# ② 売上年が無ければ「日付」から抽出
if "売上年" not in df_all.columns:
    if "日付" not in df_all.columns:
        raise ValueError("『売上年』列も『日付』列も見つかりません。")
    df_all["売上年"] = pd.to_datetime(df_all["日付"]).dt.year

# ③ 金額列を柔軟に判定（全角・半角・スペース含め対応）
normalized_cols = {col.replace("（", "(").replace("）", ")").replace(" ", ""): col for col in df_all.columns}
target_candidates = ["金額(千円)", "金額", "売上", "売上金額", "売上額"]

value_col = None
for cand in target_candidates:
    if cand in normalized_cols:
        value_col = normalized_cols[cand]
        break

if value_col is None:
    print("検出された列一覧:", list(df_all.columns))
    raise ValueError("金額列が見つかりません。列名を確認してください。")

# ④ 商品×売上年で集計
agg = (
    df_all.groupby(["商品", "売上年"], as_index=False)[value_col]
          .sum()
          .rename(columns={value_col: "金額(千円)"})
)

# ⑤ Excel出力
out_path = "売上集計表.xlsx"
with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    agg.to_excel(writer, index=False, sheet_name="Sheet1")

# ⑥ ヘッダー行を薄いグレー(#F2F2F2)
wb = load_workbook(out_path)
ws = wb["Sheet1"]
header_fill = PatternFill("solid", start_color="F2F2F2", end_color="F2F2F2")

for cell in ws[1]:
    cell.fill = header_fill

wb.save(out_path)
print("書き出し完了:", out_path)

# ⑦ Google Colab でダウンロード
try:
    files.download(out_path)
except Exception:
    print("ローカル環境では自動ダウンロードはスキップされます。ファイルは現在のフォルダ内に保存済みです。")


書き出し完了: 売上集計表.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>